## Plotting Optimization Results

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8')
from ax.service.ax_client import AxClient, ObjectiveProperties
from ax.utils.notebook.plotting import init_notebook_plotting, render
from ax.plot.contour import interact_contour
from ax.plot.diagnostic import interact_cross_validation
from ax.modelbridge.cross_validation import cross_validate

# hide future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

init_notebook_plotting()

In [ ]:
ax_client = AxClient()

# define the experiment and objectives
ax_client.create_experiment(
    parameters=[
        {"name": "PINK", "type": "range", "bounds": [0, 20], "value_type": "int"},
        {"name": "GREEN", "type": "range", "bounds": [0, 20], "value_type": "int"},
        {"name": "BLUE", "type": "range", "bounds": [0, 20], "value_type": "int"},
        {"name": "PURPLE", "type": "range", "bounds": [0, 30], "value_type": "int"},
        {"name": "SOLID_LIQUID_RATIO", "type": "range", "bounds": [0.5, 2.5], "value_type": "float", "digits": 1},
    ],
    objectives={
        "COLOR_ERR": ObjectiveProperties(minimize=True, threshold=1e-3),
        "STRAIN_ERR": ObjectiveProperties(minimize=True, threshold=1e-3),
    },
    choose_generation_strategy_kwargs={"num_initialization_trials": 0},
)

In [ ]:
running_trials = pd.read_csv(os.path.join('data', 'running_trials.csv')).astype({'PINK': int, 'GREEN': int, 'BLUE': int, 'PURPLE': int})

# separate parameters from results
X_train = running_trials.iloc[:,:5].to_dict(orient="records")
y_train = running_trials.iloc[:,5:]
y_train = [{k:v for (k,v) in zip(["COLOR_ERR", "STRAIN_ERR"], [(row[0], row[1]), (row[2], row[3])])} for row in y_train.values]

# add the previous trials to the experiment
for i in range(len(running_trials)):
    parameterization = X_train[i]
    ax_client.attach_trial(parameterization)
    ax_client.complete_trial(trial_index=i, raw_data=y_train[i])

# fit the model to the data and generate a model object
ax_client.fit_model()
model = ax_client.generation_strategy.model

### Show the effects of parameters as contour plots
This plot shows you the topology of the optimization problem for various parameters.

In [ ]:
render(interact_contour(model=model, metric_name="COLOR_ERR"))

### Plot the Model's Predictive Performance

In [ ]:
cv_results = cross_validate(model)
render(interact_cross_validation(cv_results))

### Plot Optimization Performance as a Function of Trial Number

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].scatter(running_trials.index, running_trials['COLOR_ERR_MEAN'], facecolors='none', edgecolors='C0', lw=1)
best_to_trial = [np.min(running_trials['COLOR_ERR_MEAN'][:i+1]) for i in range(len(running_trials))]
ax[0].plot(running_trials.index, best_to_trial, color='C0', lw=2)
ax[0].axvline(4.5, color='C1', ls='--', lw=1.5)
ax[0].set_xticks(running_trials.index)
ax[0].set_xticklabels(1+running_trials.index)
ax[0].set_ylabel('Color Error')
ax[0].set_xlabel('Trial Number')

ax[1].scatter(running_trials.index, running_trials['STRAIN_ERR_MEAN'], facecolors='none', edgecolors='C0', lw=1)
best_to_trial = [np.min(running_trials['STRAIN_ERR_MEAN'][:i+1]) for i in range(len(running_trials))]
ax[1].plot(running_trials.index, best_to_trial, color='C0', lw=2)
ax[1].axvline(4.5, color='C1', ls='--', lw=1.5)
ax[1].set_xticks(running_trials.index)
ax[1].set_xticklabels(1+running_trials.index)
ax[1].set_ylabel('Strain Error')
ax[1].set_xlabel('Trial Number')
plt.show()